In [1]:
import os

In [2]:
%pwd

'e:\\Data Science\\ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Data Science\\ML-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from ML_Project.constants import *
from ML_Project.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir = config.unzip_dir
            )
        
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from ML_Project import logger
from ML_Project.utils.common import get_size, read_yaml

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-19 18:33:17,730: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-19 18:33:17,742: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-19 18:33:17,754: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-19 18:33:17,756: INFO: common: created direcory at: artifacts]
[2025-01-19 18:33:17,758: INFO: common: created direcory at: artifacts/data_ingestion]
[2025-01-19 18:33:19,045: INFO: 2879261624: artifacts/data_ingestion/data.zip download! with following info: 
 Connection: close
Content-Length: 23715
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "0cd173888a5966d5f4253df2209ecc325884a694caaf2446e09b45c1845307ba"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 63E4:08DD:28148D:3A5AAD:678CF818
Accept-Ranges: bytes
Date: Sun, 19 Ja